In [1]:
from pathlib import Path
from skimage import io
import numpy as np
import napari
from os.path import exists
from brainlit.utils.session import NeuroglancerSession
from cloudvolume.exceptions import SkeletonDecodeError

/Users/thomasathey/Documents/mimlab/mouselight/docs_env/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [1]:
num = 9 #0,2,3,4,7,9
janelia = False

im_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/images/2018-08-01_" + str(num) + "_first10_quantitative.tif"
im = Path(im_path)
img = io.imread(im, plugin="tifffile")
print(f"Image shape: {img.shape}")

# read coords
if janelia:
    csv_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/voxel_coords.csv"
    coords = np.genfromtxt(csv_path, delimiter=',')
    coords = coords[10*num:10*(num+1)].astype(int)
    soma_coords = [list(coords[0,:])]
    axon_coords = [list(coords[-1,:])]
else:
    csv_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/my_points/points_" + str(num) + ".csv"
    coords = np.genfromtxt(csv_path, delimiter=',')
    coords = coords[1:].astype(int)
    coords = coords[:,1:]
    soma_coords = [list(coords[-1,:])]
    axon_coords = [list(coords[0,:])]

coords_list = list(coords)
coords_list = [list(c) for c in coords_list]
print(f"coords shape: {coords.shape}")

if num == 0:
    soma_coords += [[226, 643, 92], [233, 459, 120], [439, 500, 62]]
elif num == 7:
    soma_coords += [[335, 178, 39], [1, 418, 322]]

print(f"Soma coords: {soma_coords}")

NameError: name 'Path' is not defined

In [2]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"

ngl_sess = NeuroglancerSession(mip = 0, url = dir, url_segments=dir_segments)

num_goal = 1
num = -1
skel_id = 0

while num < num_goal:
    try:
        ngl_sess.pull_vertex_list(skel_id, [0], 1)
        print(f"Skeleton # {skel_id} exists")
        num += 1
    except SkeletonDecodeError:
        print(f"#{skel_id} invalid")
    skel_id += 1

img, bbox, vox = ngl_sess.pull_vertex_list(skel_id-1, [0,1,2,3,4,5,6,7,8,9], [50,50,15])
print(f"Image shape: {img.shape}")
coords_list = vox
soma_coords = [coords_list[0]]
axon_coords = [coords_list[-1]]

#0 invalid
#1 invalid


Downloading: 100%|██████████| 1/1 [00:00<00:00,  6.73it/s]

Skeleton # 2 exists
#3 invalid
#4 invalid
#5 invalid
#6 invalid


Skeleton # 7 exists



Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.09s/it]









Image shape: (327, 348, 70)


In [3]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img)
viewer.add_points(coords_list, face_color='green', size=8)

viewer.add_points(soma_coords, face_color='orange', size=8)
viewer.add_points(axon_coords, face_color='red', size=8)
viewer.camera.angles = [0, -90, -90]
napari.run()

Downloading: 100%|██████████| 24/24 [00:03<00:00,  6.92it/s]


# Save as zarr

In [4]:
import zarr

In [5]:
zarr_path = "/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/zarr/" + str(num) + ".zarr"

z = zarr.zeros(img.shape, chunks=img.shape, dtype=img.dtype)
z = img
zarr.save(zarr_path, z)

# Make fragments

In [6]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

In [7]:
sg = state_generation(image_path="/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/zarr/" + str(num) + ".zarr",
        ilastik_program_path="/Applications/ilastik-1.3.3post3-OSX.app/Contents/ilastik-release/run_ilastik.sh",
        ilastik_project_path="/Users/thomasathey/Documents/mimlab/mouselight/octopus_experiment/octopus_exp.ilp",
        chunk_size=img.shape,
        soma_coords=soma_coords)

In [8]:
sg.predict(data_bin="/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/zarr/misc/")

Constructing probability  image /Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/zarr/1_probs.zarr of shape (327, 348, 70)


Computing Ilastik Predictions:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
sg.compute_frags()

In [ ]:
sg.compute_soma_lbls()

In [ ]:
sg.compute_image_tiered()

In [ ]:
sg.compute_states()

In [ ]:
sg.compute_edge_weights()
viterbrain = sg.viterbrain

## Compute shortest path

In [3]:
import pickle

with open('/Users/thomasathey/Documents/mimlab/mouselight/input/images/firstn_quantitative/zarr/' + str(num) + '_viterbrain.pickle', 'rb') as handle:
    viterbrain = pickle.load(handle)

In [8]:
vb_path = viterbrain.shortest_path(axon_coords[0], soma_coords[0])
vb_path = [list(coord) for coord in vb_path]

[  0 994 995 996]
[   0   89  760  777  785 1219]


In [9]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img)
viewer.add_shapes(vb_path, shape_type='path', edge_width=2, edge_color='red', name="viterbrain")
viewer.add_shapes(coords_list, shape_type='path', edge_width=2, edge_color='green', name="truth")

Downloading: 100%|██████████| 108/108 [00:52<00:00,  2.05it/s]


<Shapes layer 'truth' at 0x167efc700>

## Load other paths and resample

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
from cloudvolume import Skeleton
import networkx as nx
from tqdm import tqdm
from brainlit.viz import Bresenham3D
import similaritymeasures

In [ ]:
def read_swc(file_path, start_pt, end_pt, switch_axes = True, reflect_y = True, factor = [1,1,1]):
    with open(file_path, "r") as f:
        swc = f.read()
    skel = Skeleton.from_swc(swc)


    verts = np.array(skel.vertices)
    if switch_axes:
        verts[:, [0,2]] = verts[:, [2,0]]
    if reflect_y:
        verts[:,1] = im_og.shape[1] - 1 - verts[:,1]
    verts = np.multiply(verts, factor)
    skel.vertices = verts

    # find path
    amins, _ = pairwise_distances_argmin_min(np.array([end_pt]), skel.vertices)
    soma_id = amins[0]

    amins, _ = pairwise_distances_argmin_min(np.array([start_pt]), skel.vertices)
    axon_id = amins[0]

    g = nx.Graph()
    num_verts = skel.vertices.shape[0]
    g.add_nodes_from(np.arange(num_verts))
    for e in tqdm(skel.edges):
        g.add_edge(e[0],e[1])
        
    try:
        graph_path = nx.shortest_path(g, source=axon_id, target=soma_id)
    except:
        graph_path = [axon_id]

    path = []
    for id in graph_path:
        path.append(skel.vertices[id, :])
    path = np.array(path)
    path = np.concatenate(([start_pt], path, [end_pt]), axis=0)

    #make mask
    skel_mask = 0*im_og

    # for edge in tqdm(skel.edges, desc='Drawing edges...'):
    #     pt1 = skel.vertices[edge[0],:].astype(int)
    #     pt2 = skel.vertices[edge[1],:].astype(int)
    #     xs, ys, zs = Bresenham3D(int(pt1[0]), int(pt1[1]), int(pt1[2]),int(pt2[0]), int(pt2[1]), int(pt2[2]))
    #     skel_mask[xs, ys, zs] = 1

    return path, skel_mask

def resample(path, spacing = 1):
    new_path = []
    for n in np.arange(path.shape[0]):
        pt1 = path[n-1:n,:]
        pt2 = path[n:n+1,:]

        new_path.append(pt1)
        dist = np.linalg.norm(pt1-pt2)

        if dist > 1:
            ts = np.arange(0, dist , 5)
            mid = np.zeros((len(ts)-1,3))
            for i,t in enumerate(ts[1:]):
                mid[i,:] = pt1 + (t/dist)*(pt2 -  pt1)
            new_path.append(mid)
    new_path.append(pt2)
    new_path = np.concatenate(new_path)
    return new_path

def sd(pts1, pts2, substantial = False, verbose=False):
    _, dists1 = pairwise_distances_argmin_min(pts1, pts2)
    _, dists2 = pairwise_distances_argmin_min(pts2, pts1)
    if verbose:
        print(dists1)
        print(dists2)
    if substantial:
        ddiv1 = np.mean(dists1[dists1 > 2])
        ddiv2 = np.mean(dists2[dists2 > 2])
        return np.mean([ddiv1, ddiv2])
    else:
        ddiv1 = np.mean(dists1)
        ddiv2 = np.mean(dists2)
        return np.mean([ddiv1, ddiv2])

In [ ]:
new_true = np.multiply(np.array(coords_list), [0.3,0.3,1])
new_true_resample = resample(new_true)
new_viterbi = np.multiply(np.array(vb_path), [0.3,0.3,1])
new_viterbi = resample(new_viterbi)

In [ ]:
app_reflect_y = False

ad_path, _ = read_swc("/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/other_traces_2/advantra_" + str(num) + ".swc",
                    axon_coords[0], soma_coords[0], reflect_y=app_reflect_y)
new_ad = np.multiply(np.array(ad_path), [0.3,0.3,1])
new_ad = resample(new_ad)

app_reflect_y = False

app_path, _ = read_swc("/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/other_traces_2/app2_" + str(num) + ".swc",
                    axon_coords[0], soma_coords[0], reflect_y=app_reflect_y)
new_app = np.multiply(np.array(app_path), [0.3,0.3,1])
new_app = resample(new_app)

#gtree
switch_axes = True
factor = [1/0.3,1/0.3,1]
reflect_y = False

gtree_path, gtree_mask = read_swc("/Users/thomasathey/Documents/mimlab/mouselight/input/images/first10_quantitative/other_traces_2/gtree_" + str(num) + "/gtree_" + str(num) + "_000.swc",
                    axon_coords[0], soma_coords[0], switch_axes=switch_axes, reflect_y=reflect_y, factor=factor)
new_gtree = np.multiply(np.array(gtree_path), [0.3,0.3,1])
new_gtree = resample(new_gtree)

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(img, scale=[0.3,0.3,1])
viewer.add_shapes(new_viterbi, shape_type='path', edge_width=2, edge_color='red', name="viterbrain")
viewer.add_shapes(new_true_resample, shape_type='path', edge_width=2, edge_color='green', name="truth")
viewer.add_shapes(new_app, shape_type='path', edge_width=2, edge_color='blue', name="app2")
viewer.add_shapes(new_ad, shape_type='path', edge_width=2, edge_color='blue', name="advantra")
viewer.add_shapes(new_gtree, shape_type='path', edge_width=2, edge_color='orange', name="gtree")

In [ ]:
print(f"Viterbi: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_viterbi)}, SSD: {sd(new_true_resample, new_viterbi, substantial = False)}")
print(f"APP2: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_app)}, SSD: {sd(new_true_resample, new_app, substantial = False)}")
print(f"Advantra: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_ad)}, SSD: {sd(new_true_resample, new_app, substantial = False)}")
print(f"gtree: Frechet: {similaritymeasures.frechet_dist(new_true_resample, new_gtree)}, SSD: {sd(new_true_resample, new_gtree, substantial = False)}")
